In [19]:
import pandas as pd
import numpy as np
import json
import warnings
import spacy
warnings.filterwarnings("ignore")
import random

import nltk
from nltk.stem import WordNetLemmatizer

import re

In [20]:
train_data = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\train.csv")
features = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\features.csv")
patient_notes = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\patient_notes.csv")

#### PN History

In [21]:
case_0 = patient_notes[patient_notes.case_num== 0]

In [22]:
case_0["pn_history"] = case_0["pn_history"].apply(lambda x: re.sub(r'''[/",()\r\n]''',' ',x))

In [23]:
new_train = train_data.copy()

In [24]:
new_train = new_train[new_train.annotation != '[]']
new_train

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']
...,...,...,...,...,...,...
14291,95333_908,9,95333,908,['Nausea'],['354 360']
14293,95333_910,9,95333,910,['lives with roomate'],['576 594']
14297,95333_914,9,95333,914,['photobia'],['274 282']
14298,95333_915,9,95333,915,['no sick contacts'],['421 437']


In [25]:
seperate_data = train_data.copy()

In [26]:
seperate_data = seperate_data[seperate_data.annotation != '[]']
seperate_data.head(3)

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']


In [27]:
seperate_data["annotation"] = seperate_data.annotation.str.replace('[','')
seperate_data["annotation"] = seperate_data.annotation.str.replace(']','')
seperate_data["location"] = seperate_data.location.str.replace('[','')
seperate_data["location"] = seperate_data.location.str.replace(']','')

seperate_data

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,'dad with recent heart attcak','696 724'
1,00016_001,0,16,1,"'mom with ""thyroid disease'",'668 693'
2,00016_002,0,16,2,'chest pressure','203 217'
3,00016_003,0,16,3,"'intermittent episodes', 'episode'","'70 91', '176 183'"
4,00016_004,0,16,4,'felt as if he were going to pass out','222 258'
...,...,...,...,...,...,...
14291,95333_908,9,95333,908,'Nausea','354 360'
14293,95333_910,9,95333,910,'lives with roomate','576 594'
14297,95333_914,9,95333,914,'photobia','274 282'
14298,95333_915,9,95333,915,'no sick contacts','421 437'


#### case 0 data

In [28]:
sep_case0 = seperate_data[seperate_data.case_num == 0]

In [29]:
sep_case0 = sep_case0.drop(["id"],axis= 1)

In [30]:
sep_case0.loc[sep_case0["annotation"] == "'dad, heart attack'","annotation"] = "'dad  heart attack'"

In [31]:
sep_case0 = (sep_case0.set_index(['pn_num',"feature_num","case_num"]) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(sep_case0.columns, axis=1))

In [32]:
sep_case0.head(3)

,case_num,pn_num,feature_num,annotation,location
0,0,16,0,'dad with recent heart attcak','696 724'
1,0,16,1,"'mom with ""thyroid disease'",'668 693'
2,0,16,2,'chest pressure','203 217'


#### Seperating start location and end location

In [33]:
sep_case0["start_location"] = sep_case0["location"].apply(lambda x: x.split()[0][1:])

In [34]:
sep_case0["end_location"] = sep_case0["location"].apply(lambda x: x.split()[-1][:-1])

#### Combinig Datasets

In [35]:
sep_combined_case0 = pd.merge(sep_case0,patient_notes,on="pn_num")


,case_num_x,pn_num,feature_num,annotation,location,start_location,end_location,case_num_y,pn_history
0,0,16,0,'dad with recent heart attcak','696 724',696,724,0,HPI: 17yo M presents with palpitations. Patien...
1,0,16,1,"'mom with ""thyroid disease'",'668 693',668,693,0,HPI: 17yo M presents with palpitations. Patien...
2,0,16,2,'chest pressure','203 217',203,217,0,HPI: 17yo M presents with palpitations. Patien...
3,0,16,3,'intermittent episodes','70 91',70,91,0,HPI: 17yo M presents with palpitations. Patien...
4,0,16,3,'episode','176 183',176,183,0,HPI: 17yo M presents with palpitations. Patien...
...,...,...,...,...,...,...,...,...,...
1198,0,2436,9,'chest palpitations','71 89',71,89,0,CC: Dillon Cleveland is a 17 year old male wit...
1199,0,2436,9,'chest palpitations','109 127',109,127,0,CC: Dillon Cleveland is a 17 year old male wit...
1200,0,2436,10,'2-3 month','50 59',50,59,0,CC: Dillon Cleveland is a 17 year old male wit...
1201,0,2436,11,'17 year old','26 37',26,37,0,CC: Dillon Cleveland is a 17 year old male wit...


In [18]:
sep_combined_case0 = sep_combined_case0.drop("case_num_x",axis = 1)

In [19]:
sep_combined_case0["New_annotation"] = sep_combined_case0.apply(lambda x: x.pn_history[int(x.start_location):int(x.end_location)],axis=1)

In [20]:
sep_combined_case0.head()

,pn_num,feature_num,annotation,location,start_location,end_location,case_num_y,pn_history,New_annotation
0,16,0,'dad with recent heart attcak','696 724',696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak
1,16,1,"'mom with ""thyroid disease'",'668 693',668,693,0,HPI: 17yo M presents with palpitations. Patien...,"mom with ""thyroid disease"
2,16,2,'chest pressure','203 217',203,217,0,HPI: 17yo M presents with palpitations. Patien...,chest pressure
3,16,3,'intermittent episodes','70 91',70,91,0,HPI: 17yo M presents with palpitations. Patien...,intermittent episodes
4,16,3,'episode','176 183',176,183,0,HPI: 17yo M presents with palpitations. Patien...,episode


In [21]:
sep_combined_case0["New_annotation"] = sep_combined_case0["New_annotation"].apply(lambda x: re.sub(r'''[/",()\r\n]''',' ',x))
sep_combined_case0["New_annotation"] = sep_combined_case0["New_annotation"].apply(lambda x: re.sub(r'''[']''','',x))
sep_combined_case0["New_annotation"] = sep_combined_case0["New_annotation"].apply(lambda x: x.strip())
sep_combined_case0

,pn_num,feature_num,annotation,location,start_location,end_location,case_num_y,pn_history,New_annotation
0,16,0,'dad with recent heart attcak','696 724',696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak
1,16,1,"'mom with ""thyroid disease'",'668 693',668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease
2,16,2,'chest pressure','203 217',203,217,0,HPI: 17yo M presents with palpitations. Patien...,chest pressure
3,16,3,'intermittent episodes','70 91',70,91,0,HPI: 17yo M presents with palpitations. Patien...,intermittent episodes
4,16,3,'episode','176 183',176,183,0,HPI: 17yo M presents with palpitations. Patien...,episode
...,...,...,...,...,...,...,...,...,...
1198,2436,9,'chest palpitations','71 89',71,89,0,CC: Dillon Cleveland is a 17 year old male wit...,chest palpitations
1199,2436,9,'chest palpitations','109 127',109,127,0,CC: Dillon Cleveland is a 17 year old male wit...,chest palpitations
1200,2436,10,'2-3 month','50 59',50,59,0,CC: Dillon Cleveland is a 17 year old male wit...,2-3 month
1201,2436,11,'17 year old','26 37',26,37,0,CC: Dillon Cleveland is a 17 year old male wit...,17 year old


#### Dropping these rows as there are spelling mistakes due to which unable to find the locations

In [22]:
sep_combined_case0["pn_history"] = sep_combined_case0["pn_history"].apply(lambda x: re.sub(r'''[/",()\r\n]''',' ',x))
sep_combined_case0["pn_history"] = sep_combined_case0["pn_history"].apply(lambda x: re.sub(r'''[']''','',x))

In [23]:
def word_lemm(sentence):
    word_list = nltk.word_tokenize(sentence)
    
    lemmaztier = WordNetLemmatizer()
    lemmatized_output = ' '.join([lemmaztier.lemmatize(w) for w in word_list])
    return(lemmatized_output)

In [24]:
sep_combined_case0["New_annotation"] = sep_combined_case0["New_annotation"].apply(word_lemm)
sep_combined_case0["pn_history"] = sep_combined_case0["pn_history"].apply(word_lemm)
sep_combined_case0.head()

,pn_num,feature_num,annotation,location,start_location,end_location,case_num_y,pn_history,New_annotation
0,16,0,'dad with recent heart attcak','696 724',696,724,0,HPI : 17yo M present with palpitation . Patien...,dad with recent heart attcak
1,16,1,"'mom with ""thyroid disease'",'668 693',668,693,0,HPI : 17yo M present with palpitation . Patien...,mom with thyroid disease
2,16,2,'chest pressure','203 217',203,217,0,HPI : 17yo M present with palpitation . Patien...,chest pressure
3,16,3,'intermittent episodes','70 91',70,91,0,HPI : 17yo M present with palpitation . Patien...,intermittent episode
4,16,3,'episode','176 183',176,183,0,HPI : 17yo M present with palpitation . Patien...,episode


In [25]:
sep_combined_case0["new_location"] = sep_combined_case0.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.pn_history),axis =1)

In [26]:
i = 0
for location in sep_combined_case0["new_location"]:
    if location == None:
        i+= 1
        
print(i)

10


In [27]:
drop_index = []
for  index,row in sep_combined_case0.iterrows():
    if row["new_location"] == None:
        drop_index.append(index)
print(drop_index)

[319, 375, 476, 477, 582, 634, 635, 777, 937, 1195]


In [28]:
sep_combined_case0 = sep_combined_case0.drop(drop_index,axis = 0)

In [29]:
sep_combined_case0["new_location"] = sep_combined_case0.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.pn_history).span(),axis =1)

In [30]:
sep_combined_case0  = sep_combined_case0.drop(['annotation', 'start_location','end_location'],axis = 1)

In [31]:
sep_combined_case0["start_location"] = sep_combined_case0.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.pn_history).start(),axis =1)
sep_combined_case0["end_location"] = sep_combined_case0.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.pn_history).end(),axis =1)

In [32]:
sep_combined_case0.head()

,pn_num,feature_num,location,case_num_y,pn_history,New_annotation,new_location,start_location,end_location
0,16,0,'696 724',0,HPI : 17yo M present with palpitation . Patien...,dad with recent heart attcak,"(663, 691)",663,691
1,16,1,'668 693',0,HPI : 17yo M present with palpitation . Patien...,mom with thyroid disease,"(638, 662)",638,662
2,16,2,'203 217',0,HPI : 17yo M present with palpitation . Patien...,chest pressure,"(197, 211)",197,211
3,16,3,'70 91',0,HPI : 17yo M present with palpitation . Patien...,intermittent episode,"(68, 88)",68,88
4,16,3,'176 183',0,HPI : 17yo M present with palpitation . Patien...,episode,"(81, 88)",81,88


In [33]:
sep_combined_case0 = sep_combined_case0.drop(index=1198)

In [34]:
row_index = []
start_location = []
end_location = []
pn_notes = set()
drop_index = []
for index,row in sep_combined_case0.iterrows():   # Iterating every row in trainn data
    if row["pn_num"] not in pn_notes:    # Verifying whether there is a change in pn_num to reinitialize start and end location lists
        start_location = []
        end_location = []
        pn_notes.add(row["pn_num"])      # Appending pn_num
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])
    else: 
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
            
            if row["start_location"] in start_location:
                if row["end_location"] >= sep_combined_case0.iloc[index -1 ,8]:
                    drop_index.append(index -1)
                else:
                    drop_index.append(index)
            elif row["end_location"] in end_location:
                if row["start_location"] <= sep_combined_case0.iloc[index -1,7]:
                    drop_index.append(index - 1)
                else:
                    drop_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])
            

In [35]:
print(drop_index)

[4, 7, 31, 32, 45, 57, 59, 75, 77, 78, 80, 87, 97, 98, 118, 128, 140, 146, 162, 164, 175, 188, 197, 231, 235, 236, 247, 250, 256, 266, 278, 290, 307, 310, 310, 313, 326, 328, 337, 348, 349, 351, 372, 389, 391, 393, 438, 440, 471, 495, 525, 534, 547, 550, 556, 575, 589, 603, 616, 689, 692, 713, 729, 732, 736, 746, 750, 820, 837, 846, 864, 873, 911, 912, 924, 948, 951, 959, 973, 979, 980, 997, 1010, 1072, 1088, 1107, 1121, 1121, 1140, 1187]


In [36]:
len(drop_index)

90

#### Dropping overlapping indexes

In [37]:
# first index
sep_combined_case0 = sep_combined_case0.drop(index=drop_index)

#### Dropping overlapping indexes identified while preparing test and train

In [38]:
[311,619,728,998,1122,1169,1173]

[311, 619, 728, 998, 1122, 1169, 1173]

In [39]:
sep_combined_case0 = sep_combined_case0.drop(index=311)

In [40]:
sep_combined_case0 = sep_combined_case0.drop(index=619)

In [41]:
sep_combined_case0 = sep_combined_case0.drop(index=728)

In [42]:
sep_combined_case0 = sep_combined_case0.drop(index=998)

In [43]:
sep_combined_case0 = sep_combined_case0.drop(index=1122)

In [44]:
sep_combined_case0 = sep_combined_case0.drop(index=1169)

In [45]:
sep_combined_case0 = sep_combined_case0.drop(index=1173)

In [46]:
sep_combined_case0[(sep_combined_case0.end_location == 137)]

,pn_num,feature_num,location,case_num_y,pn_history,New_annotation,new_location,start_location,end_location
1117,2285,3,'111 141',0,MR clevland is a 17 year old male ho presetn w...,5-6 time in the past 3 month,"(109, 137)",109,137


In [47]:
sep_combined_case0[(sep_combined_case0.end_location == 203)|(sep_combined_case0.end_location == 147)]

,pn_num,feature_num,location,case_num_y,pn_history,New_annotation,new_location,start_location,end_location
46,82,9,'126 131;143 151',0,17 yo M w no cardiac or arrhythmia PMH present...,heart racing and pounding,"(122, 147)",122,147
48,82,10,'187 209',0,17 yo M w no cardiac or arrhythmia PMH present...,over the past 3 month,"(182, 203)",182,203
262,575,2,'192 206',0,17yo M with heart pounding for few month . Fee...,chest pressure,"(189, 203)",189,203
1170,2428,4,'131 135;182 212',0,dillon cleveland is a 17yo male with no PMH th...,felt short of breath and some chest pressure a...,"(127, 203)",127,203


In [48]:
pn_Num_list = list(sep_combined_case0.pn_num.unique())

In [49]:
final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in sep_combined_case0[sep_combined_case0.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = sep_combined_case0[sep_combined_case0.pn_num == patient_number]["pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
            
            

In [51]:
import json

#### Saving into json file

In [52]:
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

#### Loading data from json

In [53]:
# Opening JSON file
f = open("sample.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [54]:
nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [55]:
train = data[:76]
validation = data[76:]

In [56]:
from spacy.tokens import DocBin
import spacy
from tqdm import tqdm
import random


In [57]:
#train data set

camp_train = create_training(train)
camp_train.to_disk("nbme_train.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 76/76 [00:00<00:00, 369.50it/s]

Skipping entity


In [58]:
#test data set

camp_train = create_training(validation)
camp_train.to_disk("nbme_validation.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 489.52it/s]


In [271]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [272]:
!python -m spacy train config.cfg --output ./output

[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     77.30    0.00    0.00    0.00    0.00
  2     200       7476.81   6468.93   47.14   55.98   40.71    0.47
  5     400      12124.13   2719.17   65.20   73.63   58.50    0.65
  7     600       3186.14   1689.51   61.51   63.45   59.68    0.62
 10     800       4840.26   1195.23   63.27   68.35   58.89    0.63
 13    1000       4318.84    911.58   66.40   66.80   66.01    0.66
 15    1200        668.99    676.48   60.94   63.14   58.89    0.61
 18    1400        784.07    566.35   63.22   63.60   62.85    0.63
 21    1600        40

[2022-03-29 14:30:43,028] [INFO] Set up nlp object from config
[2022-03-29 14:30:43,036] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-03-29 14:30:43,039] [INFO] Created vocabulary
[2022-03-29 14:30:43,042] [INFO] Finished initializing nlp object
[2022-03-29 14:30:43,519] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [99]:
2*((66.07*46.84)/(66.07+46.84))

54.81744398193251

In [100]:
2*((73.66*63.71)/(73.66+63.71))

68.32465021474849

In [59]:
test = np.array(patient_notes.loc[patient_notes.pn_num ==1146,"pn_history"])[0]
test

'pt 17y/m c/o palpitation since 2-3 months, episodes come and go, last for 3-4 minutes, no precipitating or releiving factors. last episode happened while playing basketball, pt felt chest pressure, and felt like almost passed out. pt has no chest pain, nause or vomitting, no tremors, no heat or cold intolerance, no changes in bladder or bowel habits. no changes in diet or polydypsia pt had changed college 7-8 months back, and has trouble adjusting to it, no rashes. pt has been taking 3-4 cups of coffee every day and red bull 4-5  in a week.\r\nno mood changes, no anxiety or panic attack\r\nROS - nothing significant except as above\r\nallergy - none\r\nmedication - pt takes aderral multiple times, last 2 days ago.\r\nPMH - none\r\nPSH - none\r\nSH - no smoking, occasional marijuana 5month ago, sexually active with girlfriend, uses condom, no STI\r\nFH - father had problems with heart and mother has hyperthyroid \r\n'

In [60]:
nlp = spacy.load("output/model-best")
doc = nlp(test)
for ent in doc.ents:
    print(ent.text, ent.label_,ent.start_char, ent.end_char)

17y/ Feature_11 3 7
m Feature_12 7 8
palpitation Feature_9 13 24
since 2-3 months Feature_10 25 41
come and go Feature_3 52 63
chest pressure Feature_2 182 196
felt like almost passed out Feature_4 202 229
no heat or cold intolerance Feature_5 285 312
taking 3-4 cups of coffee every day and red bull Feature_8 482 530
takes aderral Feature_6 668 681
father had problems with heart Feature_0 849 879
mother has hyperthyroid Feature_1 884 907


In [103]:
train_data = pd.read_csv("C:\\Users\\llohi\\OneDrive\\Desktop\\CAP_Stone\\train.csv")
train_data[train_data.pn_num == 1146]

,id,case_num,pn_num,feature_num,annotation,location
650,01146_000,0,1146,0,['father had problems with heart'],['849 879']
651,01146_001,0,1146,1,['mother has hyperthyroid'],['884 907']
652,01146_002,0,1146,2,['chest pressure'],['182 196']
653,01146_003,0,1146,3,['episodes come and go'],['43 63']
654,01146_004,0,1146,4,['felt like almost passed out'],['202 229']
655,01146_005,0,1146,5,"['no heat intolerance', 'no old intolerance']","['285 292;301 312', '285 287;297 312']"
656,01146_006,0,1146,6,['takes aderral'],['668 681']
657,01146_007,0,1146,7,[],[]
658,01146_008,0,1146,8,"['coffee every day', 'taking red bull']","['501 517', '482 488;522 530']"
659,01146_009,0,1146,9,['palpitation'],['13 24']
